<a href="https://colab.research.google.com/github/RanaweeraHK/Pytorch-Zero-to-Master/blob/main/Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26.4M/26.4M [00:00<00:00, 116MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 4.28MB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 58.8MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 9.96MB/s]


In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298542  [   64/60000]
loss: 2.294717  [ 6464/60000]
loss: 2.273377  [12864/60000]
loss: 2.274088  [19264/60000]
loss: 2.255930  [25664/60000]
loss: 2.221077  [32064/60000]
loss: 2.234445  [38464/60000]
loss: 2.194005  [44864/60000]
loss: 2.200304  [51264/60000]
loss: 2.168947  [57664/60000]
Test Error: 
 Accuracy: 44.6%, Avg loss: 2.164006 

Epoch 2
-------------------------------
loss: 2.174670  [   64/60000]
loss: 2.168990  [ 6464/60000]
loss: 2.107498  [12864/60000]
loss: 2.134367  [19264/60000]
loss: 2.073593  [25664/60000]
loss: 2.012853  [32064/60000]
loss: 2.054618  [38464/60000]
loss: 1.970060  [44864/60000]
loss: 1.989493  [51264/60000]
loss: 1.911824  [57664/60000]
Test Error: 
 Accuracy: 56.3%, Avg loss: 1.909118 

Epoch 3
-------------------------------
loss: 1.940919  [   64/60000]
loss: 1.912688  [ 6464/60000]
loss: 1.791062  [12864/60000]
loss: 1.852179  [19264/60000]
loss: 1.722025  [25664/60000]
loss: 1.671298  [32064/600